# 6 Chroma를 활용한 RAG 파이프라인 구축

In [4]:
! uv add -q python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

# 6-1 데이터 전처리를 위한 텍스트 분할기 설정     

In [5]:
# 문서 로더와 텍스트 분할기 임포트
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 - RAG에서 중요한 전처리 단계
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1500,    # 각 청크의 최대 문자 수 (너무 크면 정보가 희석, 너무 작으면 맥락 손실)
   chunk_overlap=200,  # 인접한 청크 간 중복되는 문자 수 (맥락 연결성 유지)
   separators=[
       "\n\n",    # 빈 줄 (문단 구분 우선)
       "\n",      # 줄바꿈 (문장 구분)
       " ",       # 공백 (단어 구분)
       ".",       # 마침표 (문장 종료)
       ",",       # 쉼표 (절 구분)
       "\u200b",  # 폭 없는 공백 (웹에서 흔히 발견)
       "\uff0c",  # 전각 쉼표 (한글 텍스트용)
       "\u3001",  # 한중일 쉼표 (동아시아 언어용)
       "\uff0e",  # 전각 마침표 (한글 텍스트용)
       "\u3002",  # 한중일 마침표 (동아시아 언어용)
       "",        # 마지막 구분자 (강제 분할)
   ],
)

# 워드 문서를 로드하고 설정한 text_splitter로 분할
# Docx2txtLoader: .docx 파일을 텍스트로 변환하는 로더
loader = Docx2txtLoader('../documents/law.docx')
# 문서를 로드하면서 동시에 청크로 분할하여 리스트로 반환
document_list = loader.load_and_split(text_splitter=text_splitter)


## 6-2 UpstageEmbedding 설정방법

In [ ]:
# 필요한 라이브러리들을 가져온다.
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수 파일(.env)을 불러온다.
load_dotenv()

# 업스테이지의 임베딩 모델을 초기화한다.
embedding = OpenAIEmbeddings(
    base_url='https://api.upstage.ai/v1/solar',
    api_key=os.getenv("UPSTAGE_API_KEY"),
    model='embedding-passage',
    check_embedding_ctx_length=False # 알 수 없는 모델에 대한 토큰 길이 확인 비활성화 (필수)
)

## 6-3 Chroma 벡터 저장소 생성

In [18]:
# 필요한 라이브러리들을 가져온다.
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
vector_store = Chroma.from_documents(
                    documents=document_list, 
                    embedding=embedding, 
                    collection_name='tax-original',
                    persist_directory="./tax-original"
                )


## 6-4 데이터 retrieval 테스트

In [19]:
question = '10억짜리 집을 2채 가지고 있을 때 세금을 얼마나 내나요?'

# k 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

retriever.invoke(question)


[Document(id='b59f8779-0cce-49b3-a0bc-a1e4326aabbc', metadata={'source': '../documents/law.docx'}, page_content='2. 공익법인등으로서 제1호에 해당하지 아니하는 경우: 제1항 각 호에 따른 세율\n\n3. 제1호 및 제2호 외의 경우: 다음 각 목에 따른 세율\n\n가. 2주택 이하를 소유한 경우: 1천분의 27\n\n나. 3주택 이상을 소유한 경우: 1천분의 50\n\n③주택분 과세표준 금액에 대하여 해당 과세대상 주택의 주택분 재산세로 부과된 세액(「지방세법」 제111조제3항에 따라 가감조정된 세율이 적용된 경우에는 그 세율이 적용된 세액, 같은 법 제122조에 따라 세부담 상한을 적용받은 경우에는 그 상한을 적용받은 세액을 말한다)은 주택분 종합부동산세액에서 이를 공제한다.<신설 2005. 12. 31., 2008. 12. 26., 2010. 3. 31.>\n\n④주택분 종합부동산세액을 계산할 때 주택 수 계산 및 주택분 재산세로 부과된 세액의 공제 등에 관하여 필요한 사항은 대통령령으로 정한다.<신설 2005. 12. 31., 2018. 12. 31., 2020. 6. 9.>\n\n⑤ 주택분 종합부동산세 납세의무자가 1세대 1주택자에 해당하는 경우의 주택분 종합부동산세액은 제1항ㆍ제3항 및 제4항에 따라 산출된 세액에서 제6항부터 제9항까지의 규정에 따른 1세대 1주택자에 대한 공제액을 공제한 금액으로 한다. 이 경우 제6항부터 제9항까지는 공제율 합계 100분의 80의 범위에서 중복하여 적용할 수 있다.<신설 2008. 12. 26., 2018. 12. 31., 2020. 8. 18., 2022. 9. 15.>\n\n⑥과세기준일 현재 만 60세 이상인 1세대 1주택자의 공제액은 제1항ㆍ제3항 및 제4항에 따라 산출된 세액에 다음 표에 따른 연령별 공제율을 곱한 금액으로 한다.<신설 2008. 12. 26., 2009. 5. 27., 2020. 8. 18., 202

## 6-5 retrieval_chain 생성

In [ ]:
# 필요한 라이브러리들을 가져온다.
from langchain_openai import ChatOpenAI
from langsmith import Client

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

llm = ChatOpenAI(model='gpt-4o-mini')

# LangChain 허브에서 검증된 RAG용 프롬프트 템플릿을 가져온다.
client = Client()
retrieval_qa_chat_prompt = client.pull_prompt("langchain-ai/retrieval-qa-chat", include_model=True)

# 검색된 문서들을 하나의 콘텍스트로 결합하는 체인을 만든다.
# stuff 방식은 모든 문서를 하나의 프롬프트에 넣는 간단한 방식이다.
combine_docs_chain = create_stuff_documents_chain(
   llm, retrieval_qa_chat_prompt
)

# retriever와 문서 결합 체인을 연결하여 최종 RAG 체인을 구성한다.
# 이제 질문을 하면 자동으로 관련 문서를 검색하고 LLM에게 전달한다.
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)


## 6-6 retrieval_chain을 활용한 답변 생성

In [22]:
ai_message = retrieval_chain.invoke({"input": question})
ai_message['answer']

'10억짜리 집을 2채 소유하고 있는 경우, 주택분 종합부동산세액은 1천분의 27의 세율이 적용됩니다. \n\n따라서, 두 집의 합계 가액은 20억이 되고, 세금은 다음과 같이 계산됩니다:\n\n**과세표준 = 20억 = 20,000,000,000 원**\n\n세액 = 과세표준 × 세율 = 20,000,000,000 × (27/1,000) = 540,000,000 원\n\n즉, 두 집에 대한 종합부동산세는 5억 4천만 원이 됩니다.'